In [ ]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import cv2
import matplotlib.pyplot as plt
import os

# **Utils**

In [ ]:
def fp_quantize(x, w, f):
    """
    fixed point post quantization.
    Args:
        x: floating point (can be list) 32 bit input
        w: bit width of the target fixed point
        f: fraction bit width of the target fixed point
    Returns:
        the desired fixed point quantized of the input x
    """
    i = w - f
    max = float(2 ** (i - 1) - 2 ** (-f))
    min = float(-2 ** (i - 1))
    n = float(2 ** f)
    xx = np.floor(x * n + 0.5) / n
    clipped = np.clip(xx, a_min=min, a_max=max)
    return clipped

In [ ]:
def FPToHex(x, w = 8, f = 7):
    """
    Converts a given Fixed point number to its Hex representation.
    Args:
        x: fixed point having w bits and f bits fraction input
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        Hex representation of input
    """
    x_fix = x
    x_fix = x_fix * pow(2, f)
    x_fix = int(x_fix)
    if(x_fix < 0):
        binary = bin(x_fix+(1<<w))
    else:
        binary = bin(x_fix)
    return hex(int(binary, 2))[2:]

def WriteFixPToFile(file_name, data_fp, w = 8, f = 7, mode = "w"):
    """
    Writes given Fixed point numbers specified by w bits width and f bits fraction to the given file name.
    Args:
        file_name: file name to save the output
        data_fp: fixed point list having w bits and f bits fraction input
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        None
    """
    with open(file_name, mode) as file:
        for i in range(data_fp.shape[0]):
            file.write(str(FPToHex(data_fp[i], w=w, f=f))+'\n')

Use below function if needed (like verifying outputs or debugging).

In [ ]:
def twos_complement(bin_num):
    """
    calculates the 2's complement of the given binary number.
    Args:
        bin_num: binary number in string format
    Returns:
        the binary representation after performing 2's complement
    """

    # Perform 2's complement on the binary number
    flipped = ''
    for bit in bin_num:
        flipped += '0' if bit == '1' else '1'
    comp_num = int(flipped, 2) + 1

    return bin(comp_num)[2:].zfill(len(bin_num))

def fixed_point_to_float(bin_num, w=16, f=14):
    """
    Converts a fixed point number given in binary representation to its decimal floating point.
    Args:
        bin_num: binary number in string format
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        decimal floating point
    """
        # Check if the number is negative
    sign_bit = int(bin_num[0])
    if sign_bit:
        bin_num = twos_complement(bin_num, w, f)

    # Split the binary number into integer and fractional parts
    if f > 0:
        if f == w:
            float_num = int(bin_num, 2) / 2**f
        else:
            int_part = int(bin_num[:-f], 2)
            frac_part = int(bin_num[-f:], 2) / 2**f
            float_num = int_part + frac_part
    else:
        int_part = int(bin_num, 2)
        float_num = int_part

    # Apply the sign to the floating point number if it was negative
    if sign_bit:
        float_num = -float_num

    return float_num

def hex_to_fixed_point_decimal(hex_value, w, f):
    """
    Converts a fixed point number given in hex representation to its decimal floating point.
    Args:
        hex_value: hex number in string format
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        decimal floating point
    """
    binary_value = bin(int(hex_value, 16))[2:]

    # # Determine the sign bit
    binary_value = binary_value.zfill(w)

    # return decimal_value
    return fixed_point_to_float(binary_value, w, f)

# **Data Preparation**

load and normalize data images:

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

display images:

In [ ]:
plt.figure(figsize=(4, 4))
for i in range(4):
    plt.subplot(2, 4, i + 1)
    plt.imshow(x_test[i], cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()

network constants:

In [ ]:
LATENT_SAPCE_DIM = 32
LATENT_SPACE_WIDTH = 4
LATENT_SAPCE_HEIGHT = 8

reshape images to one dimensional shape:

In [ ]:
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# **Training**

define the autoencoder model:

In [ ]:
input_img = keras.Input(shape=(x_train.shape[1],))
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(LATENT_SAPCE_DIM, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(x_train.shape[1], activation='sigmoid')(decoded)
autoencoder = keras.Model(input_img, decoded)

In [ ]:
autoencoder.summary()

define encoder:

In [ ]:
encoder = keras.Model(input_img, encoded)

define decoder:

In [ ]:
encoded_input = keras.Input(shape=(LATENT_SAPCE_DIM,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer)

train the model:

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
history = autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

get the output of the decoder from x_test input:

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

display the initial images and their corresponding decoded ones:

In [ ]:
n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

display the images in latent space (output of encoder):

In [ ]:
n = 10
plt.figure(figsize=(20, 8))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(encoded_imgs[i].reshape((LATENT_SPACE_WIDTH, LATENT_SAPCE_HEIGHT)).T)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

apply noise to the dataset:

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

display noisy images:

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

train the model based on noisy input and real target:

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

In [ ]:
encoder = keras.Model(input_img, encoded)

In [ ]:
encoded_input = keras.Input(shape=(LATENT_SAPCE_DIM,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer)


In [ ]:
encoded_imgs = encoder.predict(x_test_noisy)
decoded_imgs = decoder.predict(encoded_imgs)

show the noisy images and their corresponding denoised one using the trained model:

In [ ]:
n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

save the models:

In [ ]:
autoencoder.save("autoencoder.h5")
encoder.save("encoder.h5")
decoder.save("decoder.h5")

plot histogram of the weights of first layer:

In [ ]:
layer_weights = encoder.layers[1].get_weights()

# Plot the weight distribution
if layer_weights:
    weights = layer_weights[0] 

    plt.figure(figsize=(8, 6))
    plt.hist(weights.flatten(), bins=30, alpha=0.5, color='b', label='Weight Distribution')
    plt.title(f'Weight Distribution of Layer {0}')
    plt.xlabel('Weight Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()

## Quantization

post quantize input and output to 16 bits width having 14 bits fraction:

In [ ]:
x_train2 = fp_quantize(x_train, 17, 14)
x_test2 = fp_quantize(x_test, 17, 14)
x_train_noisy_2 = fp_quantize(x_train_noisy, 17, 14)
x_test_noisy_2 = fp_quantize(x_test_noisy, 17, 14)

define the quantization model using "qkeras" model:

In [ ]:
input_img = keras.Input(shape=(x_train2.shape[1],))
encoded = QDense(128, kernel_quantizer=quantized_bits(8, 1, alpha=1), bias_quantizer=quantized_bits(8, 1, alpha=1))(input_img)
encoded = QActivation(activation=quantized_relu(8, 1))(encoded)
encoded = QDense(64, kernel_quantizer=quantized_bits(8, 1, alpha=1), bias_quantizer=quantized_bits(8, 1, alpha=1))(encoded)
encoded = QActivation(activation=quantized_relu(8, 1))(encoded)
encoded = QDense(LATENT_SAPCE_DIM, kernel_quantizer=quantized_bits(16, 2, alpha=1), bias_quantizer=quantized_bits(16, 2, alpha=1))(encoded)
encoded = QActivation(activation=quantized_relu(16, 2))(encoded)

decoded = QDense(64, kernel_quantizer=quantized_bits(8, 1, alpha=1), bias_quantizer=quantized_bits(8, 1, alpha=1))(encoded)
decoded = QActivation(activation=quantized_relu(8, 1))(decoded)
decoded = QDense(128, kernel_quantizer=quantized_bits(8, 1, alpha=1), bias_quantizer=quantized_bits(8, 1, alpha=1))(decoded)
decoded = QActivation(activation=quantized_relu(8, 1))(decoded)
decoded = QDense(x_train2.shape[1], kernel_quantizer=quantized_bits(16, 2, alpha=1), bias_quantizer=quantized_bits(16, 2, alpha=1), activation='sigmoid')(decoded)

qautoencoder = keras.Model(input_img, decoded)

train the model (Quantization Aware Training):

In [ ]:
qautoencoder.compile(optimizer='adam', loss='binary_crossentropy')
history = qautoencoder.fit(x_train_noisy_2, x_train2,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_noisy_2, x_test2))

define the encoder:

In [ ]:
qencoder = keras.Model(input_img, encoded)

define the decoder:

In [ ]:
encoded_input = keras.Input(shape=(LATENT_SAPCE_DIM,))
# Retrieve the last layer of the autoencoder model
decoder_layer = qautoencoder.layers[-5](encoded_input)
decoder_layer = qautoencoder.layers[-4](decoder_layer)
decoder_layer = qautoencoder.layers[-3](decoder_layer)
decoder_layer = qautoencoder.layers[-2](decoder_layer)
decoder_layer = qautoencoder.layers[-1](decoder_layer)
# Create the decoder model
qdecoder = keras.Model(encoded_input, decoder_layer)

get the outputs of the autoencoder:

In [ ]:
encoded_imgs = qencoder.predict(x_test_noisy_2)
decoded_imgs = qdecoder.predict(encoded_imgs)

display the noisy inputs and their corresponding output using the quantized model:

In [ ]:
n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy_2[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

save the models:

In [ ]:
autoencoder.save("qautoencoder.h5")
qencoder.save("qencoder.h5")
qdecoder.save("qdecoder.h5")

##############################################################################

# **Save Weights as Fixed Point**

In [ ]:
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
qencoder = keras.models.load_model('qencoder.h5', custom_objects=co)
qencoder.compile(optimizer='adam', loss='mse')

In [ ]:
x_test_noisy_2[0]

In [ ]:
qencoder.predict(np.array([x_test2[0]]))

In [ ]:
WriteFixPToFile("input.txt", x_test2[0], 17, 14)

In [ ]:
if not os.path.exists('weights'):
    os.mkdir('weights')

In [ ]:
for i, layer in enumerate(qencoder.layers):
    weights = layer.get_weights()        
    if weights and len(weights) > 0:
        w_per_neuron = weights[0].shape[0]
        w = np.einsum("ij->ji", weights[0]).reshape(-1,)
        b = weights[1].reshape(-1,)
        separate_weights = [np.concatenate((w[c:c+w_per_neuron], b[int(c/w_per_neuron):int(c/w_per_neuron)+1]), axis=0) for c in range(0, len(w), w_per_neuron)]

        width, f = (17, 14) if layer.name == 'last' else (9, 7) 
        for n, s in enumerate(separate_weights):
            WriteFixPToFile(f"weights/layer_{i + 1}_{n + 1}_w.txt", s, width, f)